第一个任务：没有写伪代码的补充完整
第二个，比较一下自己写的和调包的时候使用KDTree和暴力求解的效率【计算一下预测的时间】

In [1]:

#1、等权、加权
#2、KNN的伪代码
import numpy as np
import pandas as pd

class KNN():
    def __init__(self, K):
        self.model = None
        self.K = K
        pass

    def fit(self, X, Y):
        '''读数据下来要存储
            knn训练过程其实是个保存训练数据的过程        
        '''
        ###将数据转化为numpy数组的形式，方便计算
        X = np.asarray(X)
        Y = np.asarray(Y)
        self.model = (X, Y)  # 封装
        return self.model

    def predict(self, X, is_weight=True):
        '''
        X: 二维列表
        is_weight: 是否采用加权投票
        return X对应的预测标签
        '''
        ### 将数据转化为numpy数组的格式
        X = np.asarray(X)
        ## 定义一个列表接收每个样本的预测结果
        result = []
        for x_index,x in enumerate(X):
#             print(f"----------预测第{x_index}条数据--------------")
            if not is_weight:
                k_neighbors_label = self._fetch_k_neighbors(x,is_weight=False)
                ### 统计每个类别出现的次数
                y_count = pd.Series(k_neighbors_label).value_counts()
                # print("y_count",y_count)
                ### 产生结果
                y_ = y_count.idxmax()
                # y_ = y_count.argmax() ##idxmax() 和 argmax 功能一样，获取最大值对应的下标索引
                result.append(y_)
            else:
                k_neighbors_label,k_neighbors_dis = self._fetch_k_neighbors(x,is_weight=True)
#                 print("label&dis\n",k_neighbors_label,k_neighbors_dis)
                # 这里先将距离转成权重
                # 权重跟距离成反比
                k_neighbors_weight = 1/(k_neighbors_dis+1/(100000*k_neighbors_dis.max()))
#                 print("归一化前权重：",k_neighbors_weight)
                # 权重归一化
                k_neighbors_weight = k_neighbors_weight/k_neighbors_weight.sum()
#                 print("归一化后权重：",k_neighbors_weight)
                label_count = {}
#                 print(k_neighbors_label)
                for index,label in enumerate(k_neighbors_label):
                    if label in label_count:
                        label_count[label] += k_neighbors_weight[index]
                    else:
                        label_count[label] = k_neighbors_weight[index]
#                 print(label_count)
                y_hat = max(label_count, key=label_count.get)
#                 print("y_hat: ",y_hat)
                result.append(y_hat)              
                
        return np.asarray(result)       
        
        

    def score(self, X, Y,is_weight=True):
        """准确率"""
        X = np.asarray(X)
        Y = np.asarray(Y)
        result = self.predict(X,is_weight=is_weight)
        return sum(Y==result)/len(Y)
        
        
        
    def _fetch_k_neighbors(self, x,is_weight=True):
        x_train, y_train = self.model 
        ## 定义一个列表用来存储每个样本的距离以及对应的标签
        # [[距离1,标签1],[距离2,标签2],[距离3,标签3]....]
        listDistance = []
        for index, i in enumerate(x_train):
            dis = np.sum((np.array(i) - np.array(x)) ** 2) ** 0.5
            listDistance.append([dis, y_train[index]])

        ## 按照dis对listDistance进行排序
        listDistance.sort()
        sort_listDistance = np.asarray(listDistance)
#         print("sort后的距离和标签：\n",sort_listDistance)
        # ## 获取取前K个最近距离的样本的标签
        k_neighbors_label = sort_listDistance[:self.K, -1]
        if not is_weight:
            return k_neighbors_label
        # ## 也可以获取前k个最近邻居的距离
        k_neighbors_dis = sort_listDistance[:self.K, 0]
        return k_neighbors_label,k_neighbors_dis



# 944570020@qq.com
# 姓名-布置作业的时间

In [2]:
T = np.array([
    [3, 104, -1],
    [2, 100, -1],
    [1, 81, -1],
    [101, 10, 1],
    [99, 5, 1],
    [98, 2, 1]])
X_train = T[:, :-1]
Y_train = T[:, -1]
x_test = [[18, 90], [50, 50]]
knn = KNN(K=5)
knn.fit(X_train, Y_train)
Y_train_hat = knn.predict(X_train,is_weight=False)
print(f"训练集预测结果：{Y_train_hat}")
print(f"训练集预测评分：{knn.score(X_train, Y_train,is_weight=False)}")
print('测试集预测结果：{}'.format(knn.predict(x_test,is_weight=False)))

训练集预测结果：[-1. -1. -1.  1.  1.  1.]
训练集预测评分：1.0
测试集预测结果：[-1.  1.]


In [3]:
T = np.array([
    [3, 104, -1],
    [2, 100, -1],
    [1, 81, -1],
    [101, 10, 1],
    [99, 5, 1],
    [98, 2, 1]])
X_train = T[:, :-1]
Y_train = T[:, -1]
x_test = [[18, 90], [50, 50]]
knn = KNN(K=5)
knn.fit(X_train, Y_train)
Y_train_hat = knn.predict(X_train,is_weight=True)
print(f"训练集预测结果：{Y_train_hat}")
print(f"训练集预测评分：{knn.score(X_train, Y_train,is_weight=True)}")
print('测试集预测结果：{}'.format(knn.predict(x_test,is_weight=True)))

训练集预测结果：[-1. -1. -1.  1.  1.  1.]
训练集预测评分：1.0
测试集预测结果：[-1.  1.]


In [4]:
print('-----------下面测试一下鸢尾花数据-----------')
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, Y = load_iris(return_X_y=True)
print(X.shape, Y.shape)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
print(x_train.shape, y_train.shape)
knn01 = KNN(K=10)
knn01.fit(x_train, y_train)
print(knn01.score(x_train, y_train))
print(knn01.score(x_test, y_test))

-----------下面测试一下鸢尾花数据-----------
(150, 4) (150,)
(120, 4) (120,)
1.0
0.9666666666666667


In [5]:
dict1 = {'a': 1000, 'b': 100}
print(max(dict1, key=dict1.get))
"""
result:
b
"""

a


'\nresult:\nb\n'

In [6]:
a = np.array([1,2,3,4,5])
a

array([1, 2, 3, 4, 5])

In [7]:
for i in a:
    print(i)

1
2
3
4
5


In [8]:
a.max()

5

In [9]:
b = 1/(a+1/(100000*a.max()))
b

array([0.999998  , 0.4999995 , 0.33333311, 0.24999988, 0.19999992])

In [10]:
b/b.sum()

array([0.43795589, 0.21897816, 0.14598549, 0.10948914, 0.08759132])

In [11]:
1/a

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ])

In [12]:
b[0]

0.999998000004

In [13]:
a = np.asarray([1,0,1,0,0])
b = np.asarray([0,0,1,0,1])
print(a==b)
print(sum(a==b)/len(a))

[False  True  True  True False]
0.6


In [14]:
np.asarray(a)

array([1, 0, 1, 0, 0])

In [15]:
len(a)

5